<a href="https://colab.research.google.com/github/teticio/aventuras-con-textos/blob/master/Amazon_Opiniones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Descargar los datos

## Download the data

In [1]:
import pickle

!wget -q -O train_text.p 'https://docs.google.com/uc?export=download&id=1-0WEtyIuvB8pxENu8WJvgYFb8aMc4F8R'
!wget -q -O train_label.p 'https://docs.google.com/uc?export=download&id=1-AKY4RWMXDg035vTJQLqUXIjYMw6Ja1t'
!wget -q -O test_text.p 'https://docs.google.com/uc?export=download&id=1-4WBoOczIwOpd-Qu5n4dINcor9wC7UUk'

train_text = pickle.load(open('train_text.p', 'rb'))
train_label = pickle.load(open('train_label.p', 'rb'))
test_text = pickle.load(open('test_text.p', 'rb'))

### -----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----

## Tu modelo va aquí

## Your model goes here

In [2]:
import re

max_len = 512
train_text = [
    re.sub(r'([^\s\w]|_)+', ' ', _).lower().split() for _ in train_text
]
train_text = [_[:max_len] + [''] * (max_len - len(_)) for _ in train_text]

In [5]:
import gensim
from gensim.models.callbacks import CallbackAny2Vec


class logger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 1
        self.loss = 0

    def on_epoch_end(self, model):
        print('#{}'.format(self.epoch), 'loss =',
              (model.get_latest_training_loss() - self.loss) /
              model.batch_words)
        self.epoch += 1
        self.loss = model.get_latest_training_loss()


embedding_model = gensim.models.Word2Vec(sentences=train_text,
                                         size=100,
                                         min_count=0,
                                         iter=20,
                                         compute_loss=True,
                                         callbacks=[logger()])
print(embedding_model)

#1 loss = 14.1401609375
#2 loss = 11.349340625
#3 loss = 11.0936203125
#4 loss = 10.284378125
#5 loss = 9.76591875
#6 loss = 9.1864625
#7 loss = 9.093525
#8 loss = 8.73450625
#9 loss = 8.348775
#10 loss = 8.42749375
#11 loss = 8.29356875
#12 loss = 7.87945
#13 loss = 8.062975
#14 loss = 8.0707
#15 loss = 8.0523625
#16 loss = 7.7583625
#17 loss = 7.7898875
#18 loss = 7.7614375
#19 loss = 7.1874625
#20 loss = 7.6908625
Word2Vec(vocab=9005, size=100, alpha=0.025)


In [6]:
import numpy as np

X_train = np.concatenate([
    np.array([embedding_model.wv.vocab[word].index
              for word in _])[np.newaxis, :] for _ in train_text
],
                         axis=0)
y_train = np.array(train_label)

In [7]:
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense
from keras.layers.embeddings import Embedding

model = Sequential()
model.add(
    Embedding(input_dim=embedding_model.wv.vectors.shape[0],
              output_dim=embedding_model.wv.vectors.shape[1],
              input_length=len(X_train[0]),
              weights=[embedding_model.wv.vectors],
              trainable=False))
model.add(Conv1D(256, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(256, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(256, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/lib/python3/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/lib/python3/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/lib/python3/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 512, 100)          900500    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 508, 256)          128256    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 101, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 97, 256)           327936    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 19, 256)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 15, 256)           327936    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
__________

In [8]:
#model.load_weights('AmazonModel.h5')
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='min'),
    ModelCheckpoint('AmazonModel.h5',
                    save_best_only=True,
                    monitor='val_loss',
                    mode='min')
]
# class_weight {'0' : ... , '1' : ... }
model.fit(X_train,
          y_train,
          validation_split=0.1,
          epochs=100,
          batch_size=512,
          callbacks=callbacks)

Train on 4491 samples, validate on 499 samples
Epoch 1/100
4491/4491 [==============================] - 3s 614us/step - loss: 0.5837 - acc: 0.7747 - val_loss: 0.4430 - val_acc: 0.8257
Epoch 2/100
4491/4491 [==============================] - 0s 79us/step - loss: 0.4096 - acc: 0.8521 - val_loss: 0.4298 - val_acc: 0.8257
Epoch 3/100
4491/4491 [==============================] - 0s 79us/step - loss: 0.3916 - acc: 0.8524 - val_loss: 0.4092 - val_acc: 0.8257
Epoch 4/100
4491/4491 [==============================] - 0s 79us/step - loss: 0.3556 - acc: 0.8537 - val_loss: 0.3657 - val_acc: 0.8257
Epoch 5/100
4491/4491 [==============================] - 0s 79us/step - loss: 0.3212 - acc: 0.8626 - val_loss: 0.3366 - val_acc: 0.8397
Epoch 6/100
4491/4491 [==============================] - 0s 80us/step - loss: 0.2915 - acc: 0.8733 - val_loss: 0.3236 - val_acc: 0.8397
Epoch 7/100
4491/4491 [==============================] - 0s 79us/step - loss: 0.2721 - acc: 0.8789 - val_loss: 0.2913 - val_acc: 0.8697


In [9]:
test_text = [
    re.sub(r'([^\s\w]|_)+', ' ', _).lower().split() for _ in test_text
]
test_text = [_[:max_len] + [''] * (max_len - len(_)) for _ in test_text]

X_test = np.concatenate([
    np.array([
        embedding_model.wv.vocab.get(word, embedding_model.wv.vocab['']).index
        for word in _
    ])[np.newaxis, :] for _ in test_text
],
                        axis=0)

model.load_weights('AmazonModel.h5')
test_pred = model.predict(X_test)

### -----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----

## Tus predicciones deberían tener este formato

- Un numpy array de 2736 probabilidades de que la etiqueta sea '1' por cada uno de las muestras de test_text
- ¡El que menor log_loss (binary_cross_entropy) consigue ganará un premio y entrará en el salón de la fama!
- Manda el fichero pickleado a teticio@gmail.com para obtener tu puntuación

## Your predictions should have this format

- A numpy array of 2736 probabilities that the label is '1' for each of the test samples _text
- The one with the smallest log_ loss (binary _cross_ entropy) will win a prize and enter the hall of fame!
- Send the pickled file to teticio@gmail.com to get your score

In [10]:
pickle.dump(test_pred, open('test_pred.p', 'wb'))

try:
    from google.colab import files  # estamos en colab?
    files.download('test_pred.p')  # descargar al ordenador local
except:
    pass

print(len(test_pred))
test_pred

2736


array([[0.85790634],
       [0.8618407 ],
       [0.20527825],
       ...,
       [0.66388637],
       [0.5482094 ],
       [0.9837644 ]], dtype=float32)